In [1]:
import datetime as dt
import time
#import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#import scipy as sp

import pyxu.abc as pxa
import pyxu.operator as pxop
import pyxu.opt.stop as pxos
#from pyxu.opt.solver.pgd import PGD

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood
import utils as ut

seed = 1  # None  # for reproducibility
if seed is None:
    seed = np.random.randint(0, 1000)
print(f"Seed: {seed}")

Seed: 1


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

### Playing around with stuff

In [3]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
#[kM + k, kM+k + M] pour la k'eme tranche
indices = [index for k in range(3) for index in range(k*3 + k + 1, k*3 + k + 3 + 1)]
print(indices)

print([testarr[(k*3 + k + 1):(k*3 + k + 3 + 1)] for k in range(0, 3)])
print(testarr[indices])

[1, 2, 3, 5, 6, 7, 9, 10, 11]
[array([9, 9, 9]), array([9, 9, 9]), array([9, 9, 9])]
[9 9 9 9 9 9 9 9 9]


In [4]:
# ne pas oublier que le vecteur de parametres theta est dans R^{M^2 + M}
testL1norm = ut.L1NormPartialReg(shape=(1,3*(3+1)), S=indices, regLambda=1)

In [5]:
print(testL1norm(testarr))
print(testL1norm.prox(testarr, tau=1))

81.0
[1. 8. 8. 8. 2. 8. 8. 8. 3. 8. 8. 8.]


In [6]:
supportIndices = [0, 4]
injection = pxop.SubSample(10, supportIndices).T

x = np.ones(len(supportIndices))

print('injection:', injection(x))

injection: [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [7]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
indices = np.array([k + k*3 for k in range(3)], dtype="int32")
print(indices)
print(testarr[indices])

[0 4 8]
[1 2 3]


In [54]:
dim1 = 2
dim2 = 3

l1normdim1 = pxop.L1Norm(dim=dim1)
l2normdim2 = pxop.L2Norm(dim=dim2)

x1 = np.ones(dim1)
x2 = 2*np.ones(dim2)
bigx = np.hstack([x1, x2])

#sumOp = pxop.Sum(arg_shape=)
normOp = pxop.hstack([l1normdim1, l2normdim2])
print(normOp(bigx))
print(l1normdim1(x1) + l2normdim2(x2))

[5.46410162]
[5.46410162]


In [ ]:
toyM = 4
ez = np.array([k + k*toyM for k in range(toyM)], dtype="int32")
print(ez)

ez2 = [index for k in range(toyM) for index in range(k*toyM + k + 1, k*toyM + k + toyM + 1)]
print(ez2)

### Generation of the source

In [3]:
path = "simulated_data/events13.csv"
beta = 5
hpL = HawkesLikelihood(path=path, beta=beta)
#hpL.plot_realization()

In [4]:
start = time.time()
lip = hpL.opA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)
#np.linalg.svd to compute the largest singular value which is = to the lipschitz constant for linear operators

Computation time of the Lipschitz constant: 0.0249
Lipschitz constant: 115.24377964750194


### Generation of the source (using multiple realizations)

In [107]:
totalProcesses = 15
beta = 5

paths = [f"simulated_data/events{i}.csv" for i in range(totalProcesses)]
allHPs = np.full(totalProcesses, None)

for i in range(totalProcesses):
    print(f"Initializing process {i+1}...")
    allHPs[i] = HawkesLikelihood(path=paths[i], beta=beta)
M = allHPs[0].M
# j = 3
# print(allHPs[j].E(np.ones(allHPs[j].E.shape[1])))

Initializing process 1...
Computing A^1 out of 8, k1=100
Computing A^2 out of 8, k2=165
Computing A^3 out of 8, k3=244
Computing A^4 out of 8, k4=116
Computing A^5 out of 8, k5=255
Computing A^6 out of 8, k6=342
Computing A^7 out of 8, k7=226
Computing A^8 out of 8, k8=353
Initializing process 2...
Computing A^1 out of 8, k1=224
Computing A^2 out of 8, k2=137
Computing A^3 out of 8, k3=296
Computing A^4 out of 8, k4=169
Computing A^5 out of 8, k5=403
Computing A^6 out of 8, k6=370
Computing A^7 out of 8, k7=150
Computing A^8 out of 8, k8=620
Initializing process 3...
Computing A^1 out of 8, k1=176
Computing A^2 out of 8, k2=154
Computing A^3 out of 8, k3=277
Computing A^4 out of 8, k4=165
Computing A^5 out of 8, k5=390
Computing A^6 out of 8, k6=343
Computing A^7 out of 8, k7=171
Computing A^8 out of 8, k8=580
Initializing process 4...
Computing A^1 out of 8, k1=116
Computing A^2 out of 8, k2=145
Computing A^3 out of 8, k3=258
Computing A^4 out of 8, k4=165
Computing A^5 out of 8, k5=3

In [108]:
# Define big forward operator A: R^{M + M^2} -> R^{K1 + ... + Kn}
bigA = pxop.vstack([allHPs[i].opA for i in range(totalProcesses)])

# Define big E operator E1(x_1, ..., x_K1) + E2(x_{K1+1}, ..., x_{K1+K2}) + ...
# E: R^{K1 + ... + Kn} -> R
bigE = pxop.hstack([allHPs[i].E for i in range(totalProcesses)])

# Define big -log(likelihood operator)
bigNegLogL = bigE * bigA

In [109]:
# print(bigE)
# print(bigNegLogL)
x = 3*np.ones(bigA.shape[1])
print(np.linalg.norm(bigA(x)))

7580.119580370313


In [83]:
start = time.time()
lip = bigA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)

Computation time of the Lipschitz constant: 0.2913
Lipschitz constant: 471.8502818691068


### Defining solvers

In [98]:
# Parameter for reconstruction
#lambda_factor = 0.1

## Parameters of the solvers
# Base
remove = True
min_iterations = 1
# Stop
eps = 1e-4
tmax = 15.0
eps_dcv = 1e-2
# PFW
ms_threshold = 0.8  # original value 0.8, the lower the more atoms will HP select
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

#lambda_ = lambda_factor * np.linalg.norm(op.T(measurements), np.infty)  # rule of thumb to define lambda
lambda_ = 5e1

In [99]:
# pfw = pyfwl.PFWLasso(
#     forwardOp=hpL.opA,
#     convexOp=hpL.E,
#     lambda_=lambda_,
#     M=hpL.M,
#     negLogL=hpL.negLogL,
#     ms_threshold=ms_threshold,
#     init_correction_prec=init_correction,
#     final_correction_prec=final_correction,
#     remove_positions=remove,
#     min_correction_steps=correction_steps,
#     show_progress=False,
# )
pfw = pyfwl.PFWLasso(
    forwardOp=bigA,
    convexOp=bigE,
    lambda_=lambda_,
    M=M,
    negLogL=bigNegLogL,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=False,
)

In [93]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [94]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


Polyatomic FW: Solving ...
diff_lipschitz constant provided.
initial value: [16431.99225718]
new indices [71]
actual new indices: 1
new indices [21 41 51 53]
actual new indices: 4
new indices [ 1  5  8 11 17 21 23 26 31 33 35 37 41 42 44 51 53 61 62 68 69 71]
actual new indices: 17
new indices [ 1  5  8 11 12 17 21 23 24 26 31 33 35 37 41 44 49 50 51 53 61 62 66 68
 69 71]
actual new indices: 5
new indices [ 1  5  8 17 21 23 26 31 33 35 37 41 44 51 53 61 62 68 69 71]
actual new indices: 0
new indices [ 1  5  8 17 21 23 26 35 41 44 51 53 61 62 68 69 71]
actual new indices: 0
new indices [ 8 21 26 35 41 44 51 53 61 68 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 41 44 51 53 69 71]
actual new i

### Evaluation of the solve

In [95]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print("Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

Final value of dual certificate:
	PFW: 12.2754
Final value of objective function:
	PFW : -333856.00


In [96]:
def supp(arr):
    return np.where(np.abs(arr) > 1e-3)[0]

#print(data_p["x"][supp(data_p["x"])])
#print(data_p["x"])
for i in range(M):
    print(f"Process {i}:", data_p["x"][(i*(M + 1)):(i*(M + 1) + M + 1)])

# Arrange intensities in a M x M matrix
intensities = np.zeros((M, M))
for i in range(M):
    intensities[i, :] = data_p["x"][(i*(M + 1) + 1):(i*(M + 1) + M + 1)]

Process 0: [16.20869806  2.67172829  0.          0.          0.          3.4173012
  0.          0.          4.23705383]
Process 1: [16.03824125  0.          2.20692696  2.70115366  0.          0.
  0.          0.          3.99975223]
Process 2: [17.02410675  0.          0.          5.38425454  0.          3.49020176
  2.99743642  0.          4.67389766]
Process 3: [16.12965592  0.          0.          0.          2.45613528  0.
  3.18014648  0.          4.13706162]
Process 4: [17.1942226   2.65738627  0.          0.          0.          6.40781069
  3.12237209  0.          4.92378265]
Process 5: [17.03262747  0.          0.          0.          2.34210788  3.17114708
  5.7570425   0.          5.89426624]
Process 6: [16.25713593  0.          0.          0.          0.          0.
  0.          2.79588443  4.08012041]
Process 7: [17.58880687  0.          0.          3.04366453  0.          3.70848635
  5.43923467  0.          8.79227192]


In [97]:
# Visualize solution
fig, ax = plt.subplots(1, 1, figsize=(5,5))
im = ax.imshow(intensities/(beta*beta), # la norme L1 est alpha/beta^2
                cmap='viridis', 
                interpolation='nearest', 
                extent=[1, M, M, 1])
fig.colorbar(im, ax=ax)  # Add a colorbar to show the scale
ax.set_title('L1 norms of the $h_{ij}$')
plt.show()

for i in range(M):
    print(f"mu{i+1}:", data_p["x"][i*(M + 1)])

print(intensities/(beta*beta))

mu1: 16.20869805784162
mu2: 16.038241250738178
mu3: 17.024106754438048
mu4: 16.129655920056077
mu5: 17.19422259942564
mu6: 17.032627473854824
mu7: 16.257135933836334
mu8: 17.588806865984836
[[0.10686913 0.         0.         0.         0.13669205 0.
  0.         0.16948215]
 [0.         0.08827708 0.10804615 0.         0.         0.
  0.         0.15999009]
 [0.         0.         0.21537018 0.         0.13960807 0.11989746
  0.         0.18695591]
 [0.         0.         0.         0.09824541 0.         0.12720586
  0.         0.16548246]
 [0.10629545 0.         0.         0.         0.25631243 0.12489488
  0.         0.19695131]
 [0.         0.         0.         0.09368432 0.12684588 0.2302817
  0.         0.23577065]
 [0.         0.         0.         0.         0.         0.
  0.11183538 0.16320482]
 [0.         0.         0.12174658 0.         0.14833945 0.21756939
  0.         0.35169088]]


In [116]:
hpL.plot_realization()